## Using SQLModel to work with database tables

# <span style="color:red">clear all output before saving: db output contains passwords! </span>

this walks through process of

 - importing modules needed
 - creates a temporary db using .env


In [ ]:

from ewxpwsdb.db.database import Session, init_db, get_db_url, get_engine
from ewxpwsdb.db.models import WeatherStation, Reading, StationType, APIResponse
from ewxpwsdb.db.importdata import import_station_file, read_station_table
db_url = get_db_url()
engine = get_engine(db_url)

## create new temp database to work with

In [ ]:

init_db(engine)

In [ ]:
station_file = '../data/test_stations.tsv'
import_station_file(station_file)


In [ ]:
from sqlmodel import select
with Session(engine) as session:
        statement = select(WeatherStation)
        results = session.exec(statement)
        stations = results.all()

len(stations)

In [ ]:
# attempt to add a duplicate
station_list = read_station_table(station_file)
dup_ws = WeatherStation.model_validate(station_list[2])
with Session(engine) as session:
    session.add(dup_ws)
    session.commit()


In [ ]:
our_table_names = ['WeatherStation', 'Reading', 'apiresponse', 'stationtype']
from sqlalchemy import inspect
inspector = inspect(engine)

db_tables = list(inspector.get_table_names())
db_tables

### Example SQL

In [ ]:
from sqlmodel import text

sql="select * from weatherstation inner join stationtype on weatherstation.station_type = stationtype.station_type where stationtype.station_type = 'ZENTRA';"
with Session(engine) as session:
    stmt = text(sql)
    results = session.exec(stmt)
    records = results.all()

len(records)


### Clean up

remove test databases

In [ ]:
# if sqlite
import re
if re.match('sqlite', get_db_url()):
    from os import remove
    remove('ewxpws.db')

In [ ]:
# if postgresl
import re
from sqlmodel import delete, text
drop_stmt = text("""drop database ewxpws""")

if re.match('postgres', get_db_url()):
    with Session(engine) as session:
        session.exec(delete(Reading))
        session.exec(delete(WeatherStation))
        session.exec(delete(StationType))

        session.commit()

    # con = engine.connect()
    # con.execute(drop_stmt)
    # con.close()


In [ ]:
session.close()
engine.dispose()